# Imports and Constants

In [173]:
import sys
from copy import deepcopy
from collections import deque
from functools import total_ordering
from heapq import *

# Define Process

In [174]:
@total_ordering
class Process:
    def __init__(self, id, arrive_time, burst_time):
        self.id = id
        self.arrive_time = arrive_time
        self.burst_time = burst_time
        self.wait_start_time = arrive_time
        self.processed_time = 0
        
    # Question guarantees only one process arrives at a time
    def __eq__(self, other):
        if other == None:
            return False
        return self.arrive_time == other.arrive_time
    
    # Sort by burst_time for SRTF
    def __lt__(self, other):
        if self.burst_time != other.burst_time:
            return self.burst_time < other.burst_time
        return self.arrive_time < other.arrive_time
        
    #for printing purpose
    def __repr__(self):
        return ('[id %d : arrive_time %d,  burst_time %d]'%(self.id, self.arrive_time, self.burst_time))

# Define Schedulers
* Input: process_list, [time_quantum (Positive Integer), alpha]
* Output_1 : Schedule list contains pairs of (time_stamp, proccess_id) indicating the time switching to that proccess_id
* Output_2 : Average Waiting Time


## First Come First Serve

In [215]:
def FCFS_scheduling(process_list):
    #store the (switching time, proccess_id) pair
    schedule = []
    current_time = 0
    waiting_time = 0
    for process in process_list:
        if(current_time < process.arrive_time):
            current_time = process.arrive_time
            
        schedule.append((current_time, process.id))
        waiting_time = waiting_time + (current_time - process.arrive_time)
        current_time = current_time + process.burst_time
    average_waiting_time = waiting_time / float(len(process_list))
    
    return schedule, average_waiting_time

In [216]:
process_list = [
    Process(0, 0, 9),
    Process(1, 1, 8),
    Process(2, 2, 2),
    Process(3, 5, 2)
]

schedule, avg = FCFS_scheduling(process_list)
print schedule
print avg

[(0, 0), (9, 1), (17, 2), (19, 3)]
9.25


## Round Robin
Scheduling process_list on round robin policy with time_quantum

In [217]:
def RR_scheduling(process_list, time_quantum ):
    process_list = deepcopy(process_list)
    n = float(len(process_list))
    
    schedule = []
    current_time = 0
    waiting_time = 0
    
    current_process = None
    queue = deque()
    
    while len(queue) > 0 or len(process_list) > 0:
        if len(queue) <= 0:
            # Handle time-gaps
            current_process = process_list[0]
            process_list = process_list[1:]
            current_time = current_process.arrive_time
            schedule.append((current_time, current_process.id, 0))
        else:
            # RR process queue
            prev_process = current_process
            current_process = queue.popleft()
            if current_process.id != prev_process.id:
                prev_process.wait_start_time = current_time
                waiting_time = waiting_time + current_time - current_process.wait_start_time
                schedule.append((current_time, current_process.id, current_time - current_process.wait_start_time))
        
        # If process finish early reclaim time, otherwise preempt at quanta
        process_time = min(
            time_quantum, 
            current_process.burst_time - current_process.processed_time)
        current_time = current_time + process_time
        
        # Process arrivals during quanta
        while len(process_list) > 0 and process_list[0].arrive_time <= current_time:
            queue.append(process_list[0])
            process_list = process_list[1:]
        
        # Re-enqueue current process if not finished
        current_process.processed_time = current_process.processed_time + process_time
        if current_process.processed_time < current_process.burst_time:
            queue.append(current_process)
    
    return (schedule, waiting_time / n)

In [218]:
process_list = [
    Process(0, 0, 9),
    Process(1, 1, 8),
    Process(2, 2, 2),
    Process(3, 5, 2)
]

schedule, avg = RR_scheduling(process_list, 2)
print schedule
print avg

[(0, 0, 0), (2, 1, 1), (4, 2, 2), (6, 0, 4), (8, 1, 4), (10, 3, 5), (12, 0, 4), (14, 1, 4), (16, 0, 2), (18, 1, 2), (20, 0, 2)]
7.5


## Shortest Remaining Time First 
Scheduling process_list on SRTF, using process.burst_time to calculate the remaining time of the current process

In [212]:
def SRTF_scheduling(process_list):
    process_list = deepcopy(process_list)
    n = float(len(process_list))
    
    schedule = []
    current_time = 0
    waiting_time = 0
    
    current_process = None
    queue = []
    
    while current_process != None or len(queue) > 0 or len(process_list) > 0:
        if current_process == None:
            if len(queue) > 0:
                current_process = heappop(queue)
                waiting_time = waiting_time + current_time - current_process.wait_start_time
            else:
                # Handle time gaps
                current_process = process_list[0]
                process_list = process_list[1:]
                current_time = current_process.arrive_time
                
            schedule.append((current_time, current_process.id))
        
        # Advance time
        current_time = current_time + 1
        current_process.burst_time = current_process.burst_time - 1

        # Process arrivals
        if len(process_list) > 0 and process_list[0].arrive_time == current_time:
            # Preempt current process if new process is shorter
            if process_list[0].burst_time < current_process.burst_time:
                current_process.wait_start_time = current_time
                heappush(queue, current_process)
                current_process = process_list[0]
                schedule.append((current_time, current_process.id))
            else:
                heappush(queue, process_list[0])   
            process_list = process_list[1:]        
        
        # Remove process if finished
        if current_process.burst_time <= 0:
            current_process = None            

    return (schedule, waiting_time / n)

In [213]:
process_list = [
    Process(0, 0, 10),
    Process(1, 3, 5),
    Process(2, 7, 1),
    Process(3, 25, 10)
]

schedule, avg = SRTF_scheduling(process_list)
print schedule
print avg

[(0, 0), (3, 1), (8, 2), (9, 0), (25, 3)]
1.75


## Shortest Job First
Scheduling using future prediction SJF without using information from process.burst_time

Let initial guess = 5 time units.

In [181]:
def SJF_scheduling(process_list, alpha):
    process_list = deepcopy(process_list)
    n = float(len(process_list))
    
    schedule = []
    current_time = 0
    waiting_time = 0
    
    current_process = None
    history = {}
    queue = []
    
    while len(queue) > 0 or len(process_list) > 0:
        if len(queue) <= 0:
            # Handle time-gaps
            current_process = process_list[0]
            process_list = process_list[1:]
            current_time = current_process.arrive_time
            schedule.append((current_time, current_process.id))
        else:
            # Find shortest job
            current_process = heappop(queue)
            waiting_time = waiting_time + current_time - current_process.wait_start_time
            schedule.append((current_time, current_process.id))
        
        # Process current procees till finish
        current_time = current_time + current_process.burst_time
        
        # Process arrivals during quanta
        while len(process_list) > 0 and process_list[0].arrive_time <= current_time:
            heappush(queue, process_list[0])
            process_list = process_list[1:]

    return (schedule, waiting_time / n)

In [182]:
process_list = [
    Process(0, 0, 10),
    Process(1, 5, 10),
    Process(2, 25, 10)
]

schedule, avg = SJF_scheduling(process_list, 0.5)
print schedule
print avg

[(0, 0), (10, 1), (25, 2)]
1.66666666667


# Read/Write Input/Output

In [183]:
def read_input(file_name):
    result = []
    with open(file_name) as f:
        for line in f:
            array = line.split()
            if (len(array)!= 3):
                print ("wrong input format")
                exit()
            result.append(Process(int(array[0]),int(array[1]),int(array[2])))
    return result

In [184]:
def write_output(file_name, schedule, avg_waiting_time):
    with open(file_name,'w') as f:
        for item in schedule:
            f.write(str(item) + '\n')
        f.write('average waiting time %.2f \n'%(avg_waiting_time))


# Main Call

In [210]:
process_list = read_input('input.txt')
#print ("printing input ----")
#print '\n'.join(map(str, process_list))

print ("simulating FCFS ----")
FCFS_schedule, FCFS_avg_waiting_time =  FCFS_scheduling(process_list)
#print '\n'.join(map(str, FCFS_schedule))
print FCFS_avg_waiting_time
write_output('FCFS.txt', FCFS_schedule, FCFS_avg_waiting_time )

print ("simulating RR ----")
RR_schedule, RR_avg_waiting_time =  RR_scheduling(process_list,time_quantum = 2)
#print '\n'.join(map(str, RR_schedule))
print RR_avg_waiting_time
write_output('RR.txt', RR_schedule, RR_avg_waiting_time )

print ("simulating SRTF ----")
SRTF_schedule, SRTF_avg_waiting_time =  SRTF_scheduling(process_list)
#print '\n'.join(map(str, SRTF_schedule))
print SRTF_avg_waiting_time
write_output('SRTF.txt', SRTF_schedule, SRTF_avg_waiting_time )

print ("simulating SJF ----")
SJF_schedule, SJF_avg_waiting_time =  SJF_scheduling(process_list, alpha = 0.5)
print '\n'.join(map(str, SJF_schedule))
print SJF_avg_waiting_time
write_output('SJF.txt', SJF_schedule, SJF_avg_waiting_time )

simulating FCFS ----
6.4375
simulating RR ----
8.5625
simulating SRTF ----
4.5
simulating SJF ----
(0, 0)
(9, 2)
(11, 3)
(13, 1)
(30, 3)
(35, 1)
(37, 2)
(43, 0)
(60, 2)
(67, 0)
(69, 1)
(72, 3)
(90, 1)
(100, 3)
(108, 2)
(117, 0)
5.4375
